In [31]:
from mysql.connector import connect, Error
from config import MYSQL_CONFIG

In [32]:
localhost = MYSQL_CONFIG['host']
user = MYSQL_CONFIG['user']
password = MYSQL_CONFIG['password']
database = MYSQL_CONFIG['database']

In [33]:
mydb = connect(
    host=localhost,
    user=user,
    password=password,
    database=database
)

In [34]:
mycursor = mydb.cursor(buffered=True)

In [35]:
mycursor.execute("SHOW TABLES")

for x in mycursor:
  print(x)

('etimad_classification_keywords',)
('etimad_classifications',)
('tender_keywords',)
('tenders',)


In [36]:
mycursor.execute("SELECT * FROM etimad_classification_keywords")

myresult = mycursor.fetchall()

for x in myresult:
  print(x)

(1, 1, 'Data Governance', 'حوكمة البيانات', datetime.datetime(2025, 7, 13, 13, 35, 52))
(2, 2, 'Cisco Networking', 'شبكات سيسكو', datetime.datetime(2025, 7, 13, 13, 38, 23))
(3, 3, 'AWS Cloud Solutions', 'حلول سحابة أمازون', datetime.datetime(2025, 7, 13, 13, 53, 29))


In [37]:
mycursor.execute("""
SELECT 
    ec.name_ar AS classification_name_ar,
    eck.keyword_ar AS keyword_ar
FROM 
    etimad_classifications AS ec
JOIN 
    etimad_classification_keywords AS eck 
    ON ec.id = eck.classification_id
ORDER BY 
    ec.unit, ec.name_ar, eck.keyword_ar
""")


myresult = mycursor.fetchall()

for x in myresult:
    print(x)

('إدارة البيانات', 'حوكمة البيانات')
('إدارة البيانات', 'شبكات سيسكو')
('خدمات السحابة', 'حلول سحابة أمازون')


In [38]:
mycursor.execute("""
SELECT *
FROM 
    tenders
""")
myresult = mycursor.fetchall()
for x in myresult:
    print(x)

(1, 'https://tenders.etimad.sa/Tender/DetailsForVisitor?STenderId=FQBfbgojPzEsnoIC5oPFew==', 'تجديد رخص أدوات إدارة وحوكمة البيانات ودليل البيانات والدعم الفني', '107247', '250639011591', 'تجديد رخص أدوات إدارة وحوكمة البيانات ودليل البيانات والدعم الفني ...عرض الأقل...', Decimal('600.00'), 'Submitted', '12 شهر', 'لا', 'منافسة عامة', 'وزارة الموارد البشرية والتنمية الاجتماعية', datetime.datetime(2025, 7, 13, 0, 0), datetime.datetime(2025, 7, 19, 9, 59), datetime.datetime(2025, 7, 19, 10, 0), None, 5, datetime.datetime(2025, 8, 14, 0, 0), datetime.datetime(2025, 9, 10, 0, 0), datetime.datetime(2025, 7, 6, 0, 0), 5, 'ديوان الوزارة بالرياض مخرج 9 مبنى رقم 15 الدور الثالث سكرتير لجنة فتح العروض', None)
(2, 'https://example.com/tender4', 'Network Upgrade with Cisco Solutions', '109002', '250740056789', 'Enhancing network infrastructure using Cisco solutions for improved reliability', Decimal('1200.00'), 'Under Evaluation', '24 months', 'نعم', 'منافسة عامة', 'وزارة الاتصالات', datetime.datet

In [39]:
mycursor.execute('''
SELECT 
    t.tender_name,
    eck.keyword_en AS keyword_name_en,
    eck.keyword_ar AS keyword_name_ar,
    ec.name_en AS classification_name_en,
    ec.name_ar AS classification_name_ar
FROM 
    tenders AS t
JOIN 
    tender_keywords AS tk ON t.id = tk.tender_id
JOIN 
    etimad_classification_keywords AS eck ON tk.keyword_id = eck.id
JOIN 
    etimad_classifications AS ec ON eck.classification_id = ec.id
ORDER BY 
    t.tender_name, ec.name_en, eck.keyword_en;
''')

myresult = mycursor.fetchall()

for x in myresult:
    print(x)

('Cloud Migration Services', 'AWS Cloud Solutions', 'حلول سحابة أمازون', 'Cloud Services', 'خدمات السحابة')
('Network Upgrade with Cisco Solutions', 'Cisco Networking', 'شبكات سيسكو', 'Data Management', 'إدارة البيانات')
('تجديد رخص أدوات إدارة وحوكمة البيانات ودليل البيانات والدعم الفني', 'Data Governance', 'حوكمة البيانات', 'Data Management', 'إدارة البيانات')


In [46]:
mycursor.execute('''
SELECT 
    eck.keyword_ar AS keyword_name_ar,
    ec.name_ar AS classification_name_ar
FROM 
    etimad_classification_keywords AS eck
JOIN 
    etimad_classifications AS ec ON eck.classification_id = ec.id
ORDER BY 
    ec.name_en, eck.keyword_en;
''')

myresult = mycursor.fetchall()
    
data = {}
for row in myresult:
    print(row)
    print(row[0])
    print(row[1])
    classification = row[1]  # ec.name_en
    keyword = row[0]         # eck.keyword_en
    if classification not in data:
        data[classification] = []
    data[classification].append(keyword)

('حلول سحابة أمازون', 'خدمات السحابة')
حلول سحابة أمازون
خدمات السحابة
('شبكات سيسكو', 'إدارة البيانات')
شبكات سيسكو
إدارة البيانات
('حوكمة البيانات', 'إدارة البيانات')
حوكمة البيانات
إدارة البيانات


In [47]:
print(data)

{'خدمات السحابة': ['حلول سحابة أمازون'], 'إدارة البيانات': ['شبكات سيسكو', 'حوكمة البيانات']}


In [ ]:
import json
print(json.dumps(data, indent=4, ensure_ascii=False))

{
    "خدمات السحابة": [
        "حلول سحابة أمازون"
    ],
    "إدارة البيانات": [
        "شبكات سيسكو",
        "حوكمة البيانات"
    ]
}


: 

In [43]:
classifications = mycursor.execute("""
    SELECT 
        ec.id AS classification_id,
        ec.unit,
        ec.name_en AS classification_name,
        GROUP_CONCAT(eck.keyword_en) AS keywords
    FROM 
        etimad_classifications ec
    LEFT JOIN 
        etimad_classification_keywords eck ON ec.id = eck.classification_id
    GROUP BY 
        ec.id, ec.unit, ec.name_en
    ORDER BY 
        ec.id
""")
classifications = mycursor.fetchall()
print(classifications)

[(1, 'Data & Innovation', 'Data Management', 'Data Governance'), (2, 'Data & Innovation', 'Data Management', 'Cisco Networking'), (3, 'Data & Innovation', 'Cloud Services', 'AWS Cloud Solutions')]


In [44]:
mycursor.execute("""
SELECT *
FROM 
    etimad_classifications
""")
myresult = mycursor.fetchall()
for x in myresult:
    print(x)

(1, 'Data & Innovation', 'Data Management', 'إدارة البيانات', 'Projects related to data handling and analysis', datetime.datetime(2025, 7, 13, 13, 35, 52))
(2, 'Data & Innovation', 'Data Management', 'إدارة البيانات', 'Projects aimed at improving network infrastructure for better connectivity and security', datetime.datetime(2025, 7, 13, 13, 38, 23))
(3, 'Data & Innovation', 'Cloud Services', 'خدمات السحابة', 'Projects related to cloud computing and services', datetime.datetime(2025, 7, 13, 13, 53, 29))


In [45]:
mycursor.execute("""
SELECT *
FROM 
    etimad_classifications
""")
myresult = mycursor.fetchall()
for x in myresult:
    print(x)

(1, 'Data & Innovation', 'Data Management', 'إدارة البيانات', 'Projects related to data handling and analysis', datetime.datetime(2025, 7, 13, 13, 35, 52))
(2, 'Data & Innovation', 'Data Management', 'إدارة البيانات', 'Projects aimed at improving network infrastructure for better connectivity and security', datetime.datetime(2025, 7, 13, 13, 38, 23))
(3, 'Data & Innovation', 'Cloud Services', 'خدمات السحابة', 'Projects related to cloud computing and services', datetime.datetime(2025, 7, 13, 13, 53, 29))


In [ ]:
id
link 
tender_name 
tender_number 
reference_number 
purpose 
document_value 
status
contract_duration 
insurance_required 
tender_type
government_entity 
last_query_date 
last_submission_date
opening_date 
evaluation_date
suspension_period 
expected_award_date 
start_date 
question_start_date 
max_query_response_time 
opening_location 
attachment 
created_at 

In [ ]:
            key_mapping = {
                "رقم_المنافسة": "tender_number",
                "اسم_المنافسة": "tender_name",
                "الرقم_المرجعي": "reference_number",
                "الغرض_من_المنافسة": "purpose",
                "قيمة_وثائق_المنافسة": "document_value",
                "حالة_المنافسة": "status",
                "مدة_العقد": "contract_duration",
                "هل_التأمين_من_متطلبات_المنافسة": "insurance_required",
                "نوع_المنافسة": "tender_type",
                "الجهة_الحكوميه": "government_entity",
                "آخر_موعد_لإستلام_الإستفسارات": "last_query_date",
                "آخر_موعد_لتقديم_العروض": "last_submission_date",
                "تاريخ_فتح_العروض": "opening_date",
                "تاريخ_فحص_العروض": "evaluation_date",
                "فترة_التوقف": "suspension_period",
                "التاريخ_المتوقع_للترسية": "expected_award_date",
                "تاريخ_بدء_الأعمال_/_الخدمات": "start_date",
                "بداية_إرسال_الأسئلة_و_الاستفسارات": "question_start_date",
                "اقصى_مدة_للاجابة_على_الاستفسارات": "max_query_response_time",
                "مكان_فتح_العرض": "opening_location",
                "Link": "link"
            }

In [16]:
mapped_tender = {
    "tender_number": "2025/04-05",
    "tender_name": "نظام تحليل البيانات وذكاء الأعمال ودعم اتخاذ القرار لقطاعات الهيئة المختلفة",
    "reference_number": "250539010015",
    "purpose": "توحيد مصادر البيانات واستخراج البيانات المرجعية والرئيسية عن طريق بناء مستودع بيانات حسب السياسات والتنظيمات الصادرة من مكتب إدارة البيانات الوطنية ...عرض الأقل...",
    "document_value": "1000.00",
    "status": "معتمدة",
    "contract_duration": "24 شهر",
    "insurance_required": "لا",
    "tender_type": "منافسة إتفاقية إطارية",
    "government_entity": "الهيئة العامة للولاية على أموال القاصرين ومن في حكمهم",
    "last_query_date": "23/07/2025 28/01/1447",
    "last_submission_date": "27/07/2025 02/02/1447 09:59 AM",
    "opening_date": "27/07/2025 02/02/1447 10:00 AM",
    "evaluation_date": "لا يوجد",
    "suspension_period": "5",
    "expected_award_date": "07/09/2025 15/03/1447",
    "start_date": "30/09/2025 08/04/1447",
    "question_start_date": "14/07/2025 19/01/1447",
    "max_query_response_time": "3",
    "opening_location": "المقر الرئيس",
    "link": "https://tenders.etimad.sa/Tender/DetailsForVisitor?STenderId=bmLJ22SP0JGprYQ3s7BkxA=="
}

In [10]:
import re
from datetime import datetime

In [11]:
def parse_arabic_datetime(value: str) -> datetime:
    """
    Extract Gregorian date and time (if exists) from Arabic Etimad string,
    convert to datetime object. Ignores Hijri date.
    """
    if not value or not isinstance(value, str):
        return None

    # Match date like 27/07/2025 and optional time like 09:59 AM
    match = re.search(r'(\d{2}/\d{2}/\d{4})(?:\s+(\d{1,2}:\d{2}\s*(?:AM|PM)?))?', value)
    if not match:
        return value

    date_part = match.group(1)
    time_part = match.group(2) or "00:00"

    # Normalize time format (e.g., "09:59 AM" or "10:00")
    try:
        if "AM" in time_part or "PM" in time_part:
            dt = datetime.strptime(f"{date_part} {time_part}", "%d/%m/%Y %I:%M %p")
        else:
            dt = datetime.strptime(f"{date_part} {time_part}", "%d/%m/%Y %H:%M")
        return dt
    except ValueError:
        return None

In [12]:
for key in tender:
    if key in ["last_query_date", "last_submission_date", "opening_date", 
               "evaluation_date", "expected_award_date", "start_date", "question_start_date"]:
        tender[key] = parse_arabic_datetime(tender[key])

In [14]:
tender

{'tender_number': '2025/04-05',
 'tender_name': 'نظام تحليل البيانات وذكاء الأعمال ودعم اتخاذ القرار لقطاعات الهيئة المختلفة',
 'reference_number': '250539010015',
 'purpose': 'توحيد مصادر البيانات واستخراج البيانات المرجعية والرئيسية عن طريق بناء مستودع بيانات حسب السياسات والتنظيمات الصادرة من مكتب إدارة البيانات الوطنية ...عرض الأقل...',
 'document_value': '1000.00',
 'status': 'معتمدة',
 'contract_duration': '24 شهر',
 'insurance_required': 'لا',
 'tender_type': 'منافسة إتفاقية إطارية',
 'government_entity': 'الهيئة العامة للولاية على أموال القاصرين ومن في حكمهم',
 'last_query_date': datetime.datetime(2025, 7, 23, 0, 0),
 'last_submission_date': datetime.datetime(2025, 7, 27, 0, 0),
 'opening_date': datetime.datetime(2025, 7, 27, 0, 0),
 'evaluation_date': 'لا يوجد',
 'suspension_period': '5',
 'expected_award_date': datetime.datetime(2025, 9, 7, 0, 0),
 'start_date': datetime.datetime(2025, 9, 30, 0, 0),
 'question_start_date': datetime.datetime(2025, 7, 14, 0, 0),
 'max_query_res

In [17]:
keys = [k for k in mapped_tender.keys() if k not in ["created_at", "attachment", "status_company"]]
for key in ["last_query_date", "last_submission_date", "opening_date", 
                "evaluation_date", "expected_award_date", "start_date", "question_start_date"]:
    mapped_tender[key] = parse_arabic_datetime(mapped_tender[key])
values = [mapped_tender[k] for k in keys]

In [18]:
placeholders = ", ".join(["%s"] * len(keys))
columns = ", ".join(f"`{k}`" for k in keys)
update_clause = ", ".join([f"`{k}`=VALUES(`{k}`)" for k in keys if k != "tender_number"])

In [21]:
sql = f"""
INSERT INTO tenders ({columns})
VALUES ({placeholders})
ON DUPLICATE KEY UPDATE {update_clause}
"""

In [22]:
print(sql)


INSERT INTO tenders (`tender_number`, `tender_name`, `reference_number`, `purpose`, `document_value`, `status`, `contract_duration`, `insurance_required`, `tender_type`, `government_entity`, `last_query_date`, `last_submission_date`, `opening_date`, `evaluation_date`, `suspension_period`, `expected_award_date`, `start_date`, `question_start_date`, `max_query_response_time`, `opening_location`, `link`)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
ON DUPLICATE KEY UPDATE `tender_name`=VALUES(`tender_name`), `reference_number`=VALUES(`reference_number`), `purpose`=VALUES(`purpose`), `document_value`=VALUES(`document_value`), `status`=VALUES(`status`), `contract_duration`=VALUES(`contract_duration`), `insurance_required`=VALUES(`insurance_required`), `tender_type`=VALUES(`tender_type`), `government_entity`=VALUES(`government_entity`), `last_query_date`=VALUES(`last_query_date`), `last_submission_date`=VALUES(`last_submission_date`), `opening_